In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split
from nltk import RegexpParser, Tree
import pandas as pd
import numpy as np

In [2]:
from vncorenlp import VnCoreNLP
annotator = VnCoreNLP("./VnCoreNLP-master/VnCoreNLP-1.1.1.jar", annotators="wseg,pos", max_heap_size='-Xmx500m')

def word_segment(text):
    annotated_text = annotator.annotate(text)['sentences'][0]
    return [dic['form'].replace('_',' ').lower() for dic in annotated_text]

In [4]:
data = pd.read_csv('data/data_labeled_updated.csv')

In [5]:
data.head()

,Unnamed: 0,NewsTitle,NewsFullContent,label
0,0,VCB ký hợp đồng vay vốn 100 triệu USD,Ông Nguyễn Hòa Bình Chủ tịch HĐQT của VCB cho ...,"[('ông', 0), ('nguyễn hoà bình chủ tịch', 0), ..."
1,1,"Kết thúc Phiên 1, 30/10: STB và PPC đang &quot...",Trong khi thị trường Châu Âu có phiên tăng điể...,"[('trong', 0), ('khi', 0), ('thị trường', 1), ..."
2,2,"Ngày 23/3, Hose đón tân binh PNJ",Kế hoạch này đã được đại diện PNJ bà Cao Thị N...,"[('kế hoạch', 1), ('này', 0), ('đã', 0), ('đượ..."
3,3,Sắp khởi công Đại học FPT,Theo đó tháng 7/2009 Đại học FPT sẽ chính thức...,"[('theo', 1), ('đó', 0), ('tháng', 1), ('7/200..."
4,4,SSI đứng đầu thị phần môi giới cổ phiếu,Trong khi chỉ có 5 công ty nắm tới trên 30 ngh...,"[('trong', 0), ('khi', 0), ('chỉ', 1), ('có', ..."


In [548]:
X=data.drop('label', axis=1)
y = data['label']

In [580]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [581]:
X_train_process = X_train['NewsFullContent_lower'].apply(lambda x: annotator.annotate(str(x)))

In [582]:
def get_postag(x):
    pos = []
    for dic in x['sentences'][0]:
        pos.append(dic['posTag'])
    return pos

In [583]:
def get_form(x):
    pos = []
    for dic in x['sentences'][0]:
        pos.append(dic['form'])
    print(pos)
    return pos

In [584]:
def flatten(A):
    rt = []
    for i in A:
        if isinstance(i,list): rt.extend(flatten(i))
        else: rt.append(i)
    return rt

In [585]:
X_train['form'] = X_train_process.apply(lambda x: get_form(x))
X_train['pos_tag'] = X_train_process.apply(lambda x: get_postag(x))

'83', 'tỷ', 'đồng', 'lợi_nhuận', 'sau', 'thuế', 'hai', 'tháng', 'đầu', 'năm', 'của', 'mwg', 'đạt', '414', 'tỷ', 'đồng', 'tăng_trưởng', '40%', 'so', 'với', 'cùng', 'kỳ', 'với', 'kết_quả', 'này', 'mwg', 'đã', 'hoàn_thành', '18%', 'kế_hoạch', 'doanh_thu', 'và', '19%', 'kế_hoạch', 'lợi_nhuận', 'sau', 'hai', 'tháng', 'đầu', 'năm', 'tóm_tắt', 'kết_quả', 'kinh_doanh', 'của', 'mwg', 'sau', 'hai', 'tháng', 'đầu', 'năm', 'nguồn', 'mwg', 'cũng', 'trong', 'tháng', 'hai', 'mwg', 'mở', 'mới', '12', 'siêu', 'trên', 'cả', 'nước', 'trong', 'đó', 'chỉ', 'có', 'một', 'siêu_thị', 'thế_giới', 'di_động', 'còn', 'lại', 'là', '5', 'siêu_thị', 'điện_máy', 'xanh', 'và', '6', 'siêu_thị', 'bách_hoá', 'xanh', 'sau', 'giai_đoạn', 'ồ_ạt', 'mở', 'mới', 'siêu_thị', 'trong', 'hai', 'tháng', 'đầu', 'năm', 'nay', 'mwg', 'chỉ', 'mở', '19', 'siêu_thị', 'thế_giới', 'di_động', 'mới', 'đến', 'cuối', 'tháng', 'hai', 'doanh_nghiệp', 'này', 'có', 'tổng_cộng', '1', '327', 'siêu_thị', 'trong', 'đó', 'chuỗi', 'thế_giới', 'di_động',

In [593]:
X_train['form_text'] = X_train['form'].apply(lambda x: (" ").join(x))

In [594]:
X_train['form_text']

29     ctcp vàng_bạc đá_quý phú_nhuận mã pnj công_bố ...
305    quá_trình thoái vốn khỏi mảng bán_lẻ dự_kiến s...
364    bidv ngày hôm_qua 15/11 đã có thông_báo chính_...
199    cụ_thể hai quỹ có liên_quan đến ssi là daiwa –...
511    ngân_hàng đấu_giá 2 khoản nợ của 2 doanh_nghiệ...
                             ...                        
71     trước đó ubcknn đồng_ý chốt tỷ lế ở hữu của nh...
106    trong quý i / 2014 doanh_thu thuần vic đạt 6 1...
270    theo đó hội_đồng_quản_trị thống_nhất thành_lập...
435    ngày 14 15/1 ngân_hàng tmcp đầu_tư và phát_tri...
102    ngân_hàng đt & pt việt_nam bidv công_bố báo_cá...
Name: form_text, Length: 521, dtype: object

In [595]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(X_train['form_text'])
feature_names = tfidf.get_feature_names()

def get_ifidf_for_words(text):
    tfidf_matrix= tfidf.transform([text]).todense()
    feature_index = tfidf_matrix[0,:].nonzero()[1]
    tfidf_scores = zip([feature_names[i] for i in feature_index], [tfidf_matrix[0, x] for x in feature_index])
    return dict(tfidf_scores)

In [596]:
X_train['ifidf_for_words'] = X_train['form_text'].apply(lambda x : get_ifidf_for_words(x))

In [597]:
def get_tfidf_for_word(word, dic):
    if word in dic.keys():
        return dic[word]
    else:
        return 0

def get_tfidf_for_docs(text, dic):
    text_df = pd.DataFrame(text)
    text_df.columns = ['word']
    tfidf_words = text_df['word'].apply(lambda x: get_tfidf_for_word(x, dic))
    return tfidf_words

In [598]:
tfidf_docs = []
for i,row in X_train[['form','ifidf_for_words']].iterrows():
    tfidf_docs.append(get_tfidf_for_docs(row['form'],row['ifidf_for_words']))
X_train['tfidf_docs'] = tfidf_docs

In [599]:
X_train.head()

,Unnamed: 0,NewsTitle,NewsFullContent_lower,form,pos_tag,form_text,ifidf_for_words,tfidf_docs
29,29,PNJ lợi nhuận cả năm tăng 17% so với 2010,ctcp vàng bạc đá quý phú nhuận mã pnj công bố ...,"[ctcp, vàng_bạc, đá_quý, phú_nhuận, mã, pnj, c...","[V, N, N, V, N, N, V, N, V, N, N, Cc, N, M, N,...",ctcp vàng_bạc đá_quý phú_nhuận mã pnj công_bố ...,"{'010': 0.11612799290761981, '023': 0.13496699...",0 0.057399 1 0.079289 2 0.07928...
305,560,FPT – Thoái vốn khỏi mảng bán lẻ dự kiến sẽ ho...,quá trình thoái vốn khỏi mảng bán lẻ dự kiến s...,"[quá_trình, thoái, vốn, khỏi, mảng, bán_lẻ, dự...","[N, V, N, V, N, V, V, R, V, E, N, M, V, R, V, ...",quá_trình thoái vốn khỏi mảng bán_lẻ dự_kiến s...,"{'108': 0.0437121243807367, '15': 0.0210558782...",0 0.075242 1 0.117735 2 0.05312...
364,619,"BIDV bán đấu giá KCN Việt Hòa - Kenmark, giá k...",bidv ngày hôm qua 15/11 đã có thông báo chín...,"[bidv, ngày, hôm_qua, 15/11, đã, có, thông_báo...","[V, N, N, M, R, V, N, A, E, N, V, V, N, V, N, ...",bidv ngày hôm_qua 15/11 đã có thông_báo chính_...,"{'10': 0.017223550646880265, '11': 0.023037794...",0 0.244274 1 0.109267 2 0.04976...
199,399,SSI đăng ký mua 5 triệu cổ phiếu của DHC,cụ thể hai quỹ có liên quan đến ssi là daiwa...,"[cụ_thể, hai, quỹ, có, liên_quan, đến, ssi, là...","[A, M, N, V, V, E, N, V, N, CH, V, N, N, N, Nb...",cụ_thể hai quỹ có liên_quan đến ssi là daiwa –...,"{'01': 0.09777566697234044, '06': 0.0998546827...",0 0.054201 1 0.054419 2 0.12756...
511,770,BIDV đấu giá nợ và tài sản tổng giá khởi điểm ...,ngân hàng đấu giá 2 khoản nợ của 2 doanh ngh...,"[ngân_hàng, đấu_giá, 2, khoản, nợ, của, 2, doa...","[N, V, M, N, N, E, M, N, E, N, N, M, M, M, Nu,...",ngân_hàng đấu_giá 2 khoản nợ của 2 doanh_nghiệ...,"{'10': 0.015970561146578535, '118': 0.04272266...",0 0.062353 1 0.230460 2 0.00000...


In [600]:
X_train = X_train.reset_index()

In [603]:
def combine_feature(X):
    features = []
    for i in range(len(X_train)):
        form = pd.DataFrame(X_train['form'][i])
        pos_tag = pd.DataFrame(X_train['pos_tag'][i])
        tfidf_docs = pd.DataFrame(X_train['tfidf_docs'][i])
        df_col_merged = pd.concat([form, pos_tag,tfidf_docs], axis=1)
        feature = df_col_merged.values.tolist()
        features.append(feature)
    X['features'] = features 

In [604]:
combine_feature(X_train)

In [605]:
X_train['features']

0      [[ctcp, V, 0.05739876783897788], [vàng_bạc, N,...
1      [[quá_trình, N, 0.07524175082506061], [thoái, ...
2      [[bidv, V, 0.24427389465937988], [ngày, N, 0.1...
3      [[cụ_thể, A, 0.05420111420498017], [hai, M, 0....
4      [[ngân_hàng, N, 0.062353015264098165], [đấu_gi...
                             ...                        
516    [[trước, E, 0.15261141740424186], [đó, P, 0.04...
517    [[trong, E, 0.10188285734481316], [quý, N, 0.3...
518    [[theo, V, 0.01641649536942139], [đó, P, 0.016...
519    [[ngày, N, 0.019647684976328426], [14, M, 0.02...
520    [[ngân_hàng, N, 0.0564885536401534], [đt, V, 0...
Name: features, Length: 521, dtype: object